<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Langchain_NL2SQL_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Install required libraries for the project
!pip install langchain_openai langchain_community langchain pymysql chromadb -q

# Explanation of each library:
# - langchain_openai: Library for integrating with OpenAI's APIs.
# - langchain_community: Additional tools and integrations for LangChain.
# - langchain: Core framework for building advanced data workflows.
# - pymysql: Python library for connecting to MySQL databases.
# - chromadb: A database for managing and searching high-dimensional vectors.
# - -q: Quiet mode to suppress installation output.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.0 MB/s eta 0:00:00
 

###Define Database Connection Parameters

This step sets up the necessary credentials to connect to a MySQL database.
- **db_user**: Username used to authenticate the database connection.
- **db_password**: Password corresponding to the database user.
- **db_host**: The host or endpoint where the database is hosted (can be local or on the cloud).
- **db_name**: The name of the specific database to interact with.




https://www.mysqltutorial.org/getting-started-with-mysql/mysql-sample-database/

In [3]:
# username for the database
db_user = "*****"

# password for the database
db_password = "******"

# host address of the database
db_host = "*****.eu-west-3.rds.amazonaws.com"

# name of the database
db_name = "classicmodels"


##Building a basic NL2SQL model

### Step: Connect to the SQL Database

This step establishes a connection to a MySQL database using the `SQLDatabase` utility from `langchain_community`.
The connection is defined via a formatted URI containing the database type (`mysql`), driver (`pymysql`), and credentials (`user`, `password`, `host`, and `database name`).


In [4]:
from langchain_community.utilities.sql_database import SQLDatabase

# connect to the SQL database using the SQLDatabase utility
# the connection uses MySQL with PyMySQL as the driver
# database connection parameters (user, password, host, database name) are passed via a formatted URI
# options for customizing table metadata (like sample rows or included tables) are commented out below

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

### Step: Inspect the Database Connection and Metadata

1. **Database Dialect**: The first line prints the dialect of the database (e.g., `mysql`), which indicates the type of database system being used.
2. **Usable Table Names**: The second line retrieves and prints a list of table names available in the connected database.
3. **Table Metadata**: The third line provides detailed metadata about each table, such as column names, data types, and any custom table info included during initialization.

These outputs are helpful for understanding the structure and capabilities of the connected database.


In [5]:
# print the database dialect being used (e.g., MySQL, PostgreSQL, etc.)
print(db.dialect)

# print the list of table names in the database that can be accessed
print(db.get_usable_table_names())

# print detailed metadata information for each table in the database
print(db.table_info)

mysql
['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']

CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmit

### Step: Set Environment Variables for API Keys and Tracing

1. **OpenAI API Key**:
   - The `OPENAI_API_KEY` environment variable is used to authenticate requests made to OpenAI's services.
   - Replace the placeholder with your actual API key.

2. **LangChain Tracing**:
   - The `LANGCHAIN_TRACING_V2` environment variable enables LangChain's tracing functionality.
   - Tracing is useful for debugging and monitoring the execution of chains and tools.

3. **LangChain API Key**:
   - The `LANGCHAIN_API_KEY` is required to access certain features or services offered by LangChain.


In [6]:
import os
# set the OpenAI API key for authentication with OpenAI services
os.environ["OPENAI_API_KEY"] = "*****************************************************"
# enable LangChain's tracing functionality for debugging or monitoring
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# set the LangChain API key for accessing LangChain services
os.environ["LANGCHAIN_API_KEY"] = "*********************************************"


### Step: Generate SQL Query from Natural Language

1. **Initialize the Language Model (LLM)**:
   - The `ChatOpenAI` class is used to initialize OpenAI's GPT-3.5-turbo model with a temperature of `0`. A temperature of `0` ensures that the model’s responses are deterministic (i.e., always the same for a given input).

2. **Create SQL Query Chain**:
   - The `create_sql_query_chain` function is used to link the LLM with the database connection (`db`). This chain transforms natural language questions into SQL queries.

3. **Invoke the Query**:
   - The `invoke` method is called with a natural language question (`"what is price of 1968 Ford Mustang"`), and it generates an SQL query based on the input.

4. **Print the Query**:
   - The final SQL query is printed to the console.

This code effectively translates a user's natural language question into an SQL query, which can then be executed on the database to retrieve the desired information.


In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain


# initialize the language model (LLM) from OpenAI using GPT-3.5-turbo with a temperature of 0 for deterministic outputs
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# create the SQL query chain that takes the LLM and the database connection as input
generate_query =create_sql_query_chain(llm, db)

# invoke the query chain with a natural language question to be converted into an SQL query
query= generate_query.invoke({"question": "what is price of `1968 Ford Mustang`"})

# print the generated SQL query
print(query)

SELECT `buyPrice`, `MSRP`
FROM products
WHERE `productName` = '1968 Ford Mustang'
LIMIT 1;


### Step: Execute the SQL Query

1. **Initialize QuerySQLDataBaseTool**:
   - The `QuerySQLDataBaseTool` class from LangChain is used to execute SQL queries on the connected database (`db`). This tool allows you to run the SQL queries generated by the language model.

2. **Execute the Query**:
   - The `invoke` method is called with the SQL query (`query`), which was generated in the previous step. This executes the query on the connected database and retrieves the results.

The tool bridges the gap between natural language processing (NLP) and SQL query execution, providing a seamless way to query the database using natural language input.


In [8]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)

# invoke the tool to execute the generated SQL query on the connected database
# this will return the result of the query
execute_query.invoke(query)

"[(Decimal('95.34'), Decimal('194.57'))]"

In [9]:
# create a chain by linking the query generation step and the query execution step
chain = generate_query | execute_query

# invoke the chain with the natural language question "How many products are there"
# this will generate the SQL query and then execute it on the connected database
chain.invoke({"question": "How many products are there"})

'[(110,)]'

This step is helpful for debugging and verifying the prompt before execution.

In [10]:
# retrieve the list of prompts used by the chain and access the first prompt
# the 'pretty_print()' method is called to display the prompt in a more readable format
chain.get_prompts()[0].pretty_print()


You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S


This step defines and executes a processing chain for answering a user's question by generating an SQL query, executing it, and rephrasing the results into a user-friendly answer

In [11]:
# Import necessary modules
from operator import itemgetter

# Import classes for handling output parsing and prompt templating
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Create a PromptTemplate object that structures how the question, query, and result should be formatted
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}  # The user's question
SQL Query: {query}      # The SQL query generated to answer the question
SQL Result: {result}    # The result of executing the SQL query
Answer: """             # The final output answer to be generated
)

# Create a rephrasing pipeline that takes the SQL result and formats the answer using an LLM (large language model)
rephrase_answer = answer_prompt | llm | StrOutputParser()

# Chain the components together: first generating the query, then executing it, and finally rephrasing the result
chain = (
    RunnablePassthrough.assign(query=generate_query)        # Assign the query generation step
    .assign(result=itemgetter("query") | execute_query)     # Assign the execution of the SQL query
    | rephrase_answer                                       # Assign the rephrasing step that formats the answer
)

# Invoke the chain with the user's question about the most recent order
chain.invoke({"question": "Which order was placed most recently?"})


'The most recent order placed was order number 10424 on May 31, 2005.'

### Few-shot prompt:
 - Understanding User Intent: By showing the model how to translate a question into a query, you're teaching it to understand the user's intent and generate appropriate queries.

 - Providing Structure: It helps the model understand which SQL syntax and clauses (e.g., SELECT, WHERE, LIMIT) to use when forming a query.

- Reducing Ambiguity: The examples help reduce any ambiguity in generating queries, ensuring that the model provides accurate and well-structured queries for similar user inputs.


In [12]:
# Examples of user queries with corresponding SQL queries for training the model
examples = [
    # Query to list customers from France with a credit limit above 20,000
    {
        "input": "Can you list all customers in France who have a credit limit greater than 20,000?",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    # Query to find the highest payment made by any customer
    {
        "input": "What is the highest payment amount made by a customer?",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    # Query to show product details for products in the 'Motorcycles' category
    {
        "input": "Please show product details for all items in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    # Query to retrieve names of employees who report to employee number 1002
    {
        "input": "Which employees report to employee number 1002? Please provide their names.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    # Query to list products that have less than 7000 units in stock
    {
        "input": "Can you list all products with less than 7000 units in stock?",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    # Query to fetch the prices of the '1968 Ford Mustang'
    {
        'input': "What is the price of the '1968 Ford Mustang'?",
        'query': "SELECT `buyPrice`, `MSRP` FROM products WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]


### Step Explanation: Formatting SQL Queries with Few-Shot Learning

1. **Chat Prompt Template**:  
   A template is created to structure the conversation, where the human asks a question, and the AI responds with the SQL query.

2. **Few-Shot Learning**:  
   The model is provided with a few examples of user questions and their corresponding SQL queries to help it learn the format.

3. **Query Generation**:  
   Using the `few_shot_prompt.format()`, the model generates a SQL query for a new user input based on the examples.

This method trains the model to convert natural language questions into SQL queries.


In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate, PromptTemplate

# Creating a prompt template for a simple chat format where the 'human' asks a question and the 'ai' generates a SQL query
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),  # User asks a question, followed by the SQL query
        ("ai", "{query}"),  # The AI generates the SQL query based on the question
    ]
)

# Creating a few-shot prompt that includes example prompts to show the model how to generate SQL queries
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,  # The example prompt template
    examples=examples,  # A list of example input-output pairs
    input_variables=["input"],  # The input variable that will be used in the format
)

# Print separator for better readability
print("-----------")

# Use the few-shot prompt to format a new input question and generate the corresponding SQL query
print(few_shot_prompt.format(input1="How many products are there?"))


-----------
Human: Can you list all customers in France who have a credit limit greater than 20,000?
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: What is the highest payment amount made by a customer?
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: Please show product details for all items in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: Which employees report to employee number 1002? Please provide their names.
SQLQuery:
AI: SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;
Human: Can you list all products with less than 7000 units in stock?
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: What is the price of the '1968 Ford Mustang'?
SQLQuery:
AI: SELECT `buyPrice`, `MSRP` FROM products WHERE `productName` = '1968 Ford Mustang' LIMIT 1;


### Output Explanation

The output shows a conversation where the user asks a question, and the model generates an appropriate SQL query based on the input. For each user query, the model uses the examples it has learned to form a structured SQL query. The output demonstrates how the model processes the input and responds with the correct SQL syntax, including examples like retrieving customer data, payment amounts, and product details.

Example:
- **Input**: "How many products are there?"
- **Generated SQL Query**: "SELECT COUNT(*) FROM products;"


### Step: Dynamic few-shot example selection

In this step, dynamic few-shot example selection is implemented using semantic similarity. The Chroma vector store is initialized to store example embeddings, and any previous data is cleared. The SemanticSimilarityExampleSelector uses OpenAI embeddings to convert input queries and examples into vectors, then selects the top k most relevant examples based on similarity. This allows for adaptive selection of examples based on the user’s input, improving the model’s response

In [14]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# Initialize Chroma vector store
vectorstore = Chroma()

# Delete any existing collections in the vectorstore
vectorstore.delete_collection()

# Set up example selector with semantic similarity
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,  # List of examples
    OpenAIEmbeddings(),  # Embedding model for text
    vectorstore,  # Vector store for embeddings
    k=2,  # Number of examples to return
    input_keys=["input"],  # Key for input text
)


example_selector.select_examples({"input": "How many employees?"})


<ipython-input-14-cdaac05322f0>:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma()


[{'input': 'Which employees report to employee number 1002? Please provide their names.',
  'query': 'SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;'},
 {'input': 'Can you list all customers in France who have a credit limit greater than 20,000?',
  'query': "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"}]

Dynamic Example Selection: The FewShotChatMessagePromptTemplate replaces the static examples argument with example_selector. Instead of using pre-defined examples, it dynamically selects the most relevant examples at runtime based on the user's input.

In [17]:
# Create a FewShotChatMessagePromptTemplate using dynamic example selection
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,  # The base structure of each example (human/AI interaction)
    example_selector=example_selector,  # Dynamically selects the most relevant examples using the selector
    input_variables=["input", "top_k"],  # Input variables required for formatting (query and number of examples)
)

# Format the prompt using the given input query ("How many products are there?")
# The `example_selector` dynamically retrieves the most relevant examples to construct the final prompt
print(few_shot_prompt.format(input="How many products are there?"))


Human: Can you list all products with less than 7000 units in stock?
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Please show product details for all items in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';


Final prompts : customizing prompts

This step customizes a prompt for generating accurate SQL queries by combining a system message with dynamic few-shot examples and the user's input. The system message sets the context, the few-shot prompt provides relevant examples, and the human input specifies the question, ensuring the model generates precise and context-aware SQL queries.

In [18]:
# Define a final prompt that integrates a system message, dynamic few-shot examples, and a human input.
final_prompt = ChatPromptTemplate.from_messages(
    [
        # System-level instruction providing context for the task and relevant table information.
        ("system", "Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),

        # Include the dynamically selected few-shot examples.
        few_shot_prompt,

        # Incorporate the user's question as the final input.
        ("human", "{input}"),
    ]
)

# Format the final prompt with a specific user query ("How many products are there?") and table info.
print(final_prompt.format(input="How many products are there?", table_info="some table info"))


System: Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: Can you list all products with less than 7000 units in stock?
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Please show product details for all items in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?


The SQL query chain is built by combining the language model with the final prompt and the database connection.
The pipeline uses multiple steps: it generates a query, executes it, and rephrases the output for better readability.

In [19]:
# Generate a chain for querying the database based on user questions

# Create the SQL query chain using the defined language model (llm), database (db), and final_prompt
generate_query = create_sql_query_chain(llm, db, final_prompt)

# Build a pipeline:
# 1. Assigns the SQL query generation to `query`.
# 2. Executes the SQL query to retrieve the result using the `execute_query` tool.
# 3. Passes the SQL query result to the `rephrase_answer` prompt for better formatting and clarity.
chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

# Run the chain with a new question and generate the answer
chain.invoke({"question": "How many employees work in the Sales department?"})


'There are 20 employees who work in the Sales department.'

### Step: Dynamic relevant table selection

This step processes a CSV file containing database table information and formats it into a readable string for documentation purposes. It reads the table names and descriptions, combines them into a structured format, and outputs the details. This is useful for providing database schema context to models or for documentation purposes.


In [23]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("database_table_descriptions.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)

Table Name:productlines
Table Description:Stores information about the different product lines offered by the company, including a unique name, textual description, HTML description, and image. Categorizes products into different lines.

Table Name:products
Table Description:Contains details of each product sold by the company, including code, name, product line, scale, vendor, description, stock quantity, buy price, and MSRP. Linked to the productlines table.

Table Name:offices
Table Description:Holds data on the company's sales offices, including office code, city, phone number, address, state, country, postal code, and territory. Each office is uniquely identified by its office code.

Table Name:employees
Table Description:Stores information about employees, including number, last name, first name, job title, contact info, and office code. Links to offices and maps organizational structure through the reportsTo attribute.

Table Name:customers
Table Description:Captures data on cus

This step identifies potentially relevant SQL tables for a user's query by leveraging the provided database schema descriptions (table_details) and extracting relevant table names.



In [28]:
# Define a prompt to instruct the language model to extract relevant SQL table names
table_details_prompt = f"""Return the names of ALL the SQL tables that might be relevant to the user question. \
The tables are:

{table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

# Create an extraction chain to process the prompt and extract relevant table names
# - `Table` defines the schema for the expected output (table names in this case)
# - `llm` is the language model that will process the input
# - `system_message` contains the table description and the instructions for the model
table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)

# Invoke the chain with the user query to identify relevant tables
# The input specifies the user's question about retrieving customer details and their order count
tables = table_chain.invoke({"input": "give me details of customer and their order count"})

# Output the relevant table names
tables


[Table(name='customers'), Table(name='orders')]

In [29]:
# Extract table names from a list of `Table` objects
def get_tables(tables: List[Table]) -> List[str]:
    return [table.name for table in tables]

# Chain to select relevant tables based on the user query
select_table = (
    {"input": itemgetter("question")}
    | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
    | get_tables
)

# Run the chain with the user's query
select_table.invoke({"question": "give me details of customer and their order count"})


['customers', 'orders']

In [34]:
# Define a chain to process a query and produce the desired result
chain = (
    # Step 1: Use `select_table` to identify relevant table names for the query
    RunnablePassthrough.assign(table_names_to_use=select_table)
    |
    # Step 2: Generate a query based on the user input and execute it
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    |
    # Step 3: Rephrase the query's result for user-friendly output
    rephrase_answer
)

# Invoke the chain with a specific question
chain.invoke({"question": "How many customers with order count more than 5"})


'There are 2 customers with an order count greater than 5. Customer number 124 has 17 orders and customer number 141 has 26 orders.'

In [33]:
chain.invoke({"question": "Can you list their names?"})

"The names of the customers from France with a credit limit greater than $20,000 are: Atelier graphique, La Rochelle Gifts, Saveley & Henriot, Co., Daedalus Designs Imports, La Corne D'abondance, Co., Mini Caravy, Alpha Cognac, Lyon Souveniers, Auto Associés & Cie., Marseille Mini Autos, Reims Collectables, and Auto Canal+ Petit."

### Final step : adding memory to the chatbot


Adding memory to the chatbot enables it to retain context from previous interactions, which is crucial for answering follow-up questions or handling multi-turn conversations effectively. In the context of a database-related chatbot, memory allows the system to remember key information




In [35]:
# Define a final prompt for generating SQL queries with memory support
final_prompt = ChatPromptTemplate.from_messages(
    [
        # System message providing instructions and context for query generation
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and should be considered while answering follow up questions"),

        # Dynamically selected few-shot examples to guide the model in query generation
        few_shot_prompt,

        # Placeholder for storing previous messages to add memory and context
        MessagesPlaceholder(variable_name="messages"),

        # Human input representing the current question/query
        ("human", "{input}"),
    ]
)

# Format the prompt with specific details:
# - `input`: The user's current question ("How many products are there?")
# - `table_info`: Information about database tables ("some table info")
# - `messages`: An empty list indicating no prior conversation history (for fresh start)
print(final_prompt.format(input="How many products are there?", table_info="some table info", messages=[]))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and should be considered while answering follow up questions
Human: Can you list all products with less than 7000 units in stock?
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Please show product details for all items in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?


This code integrates memory with the chain and assigns necessary operations for query generation and execution.

In [37]:
from langchain.memory import ChatMessageHistory  # Importing memory functionality to track conversation history
history = ChatMessageHistory()  # Initializing the message history for storing past interactions

# Creating the SQL query generation chain using the final prompt
generate_query = create_sql_query_chain(llm, db, final_prompt)

# Creating a chain of tasks where each task assigns a specific operation
chain = (
    RunnablePassthrough.assign(table_names_to_use=select_table) |  # Assigning the relevant tables based on the user's query
    RunnablePassthrough.assign(query=generate_query).assign(  # Generating the query from the LLM
        result=itemgetter("query") | execute_query  # Extracting and executing the generated query
    ) |
    rephrase_answer  # Rephrasing the result of the query before sending it back to the user
)


In [38]:
question = "How many cutomers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages})
response

'There are 2 customers with an order count greater than 5. Customer 124 has 17 orders and customer 141 has 26 orders.'

This code stores the interaction between the user and AI into the chat history. The add_user_message method records the user's input, and add_ai_message logs the AI's response, maintaining a history of the conversation for future context, especially for follow-up questions.

In [40]:
history.add_user_message(question)  # Adds the user's question to the chat history
history.add_ai_message(response)  # Adds the AI's response to the chat history


In [41]:
history.messages

[HumanMessage(content='How many cutomers with order count more than 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='There are 2 customers with an order count greater than 5. Customer 124 has 17 orders and customer 141 has 26 orders.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How many cutomers with order count more than 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='There are 2 customers with an order count greater than 5. Customer 124 has 17 orders and customer 141 has 26 orders.', additional_kwargs={}, response_metadata={})]

In [42]:
response = chain.invoke({"question": "Can you list there names?","messages":history.messages})
response

"The names of the customers who have placed more than 5 orders are 'Mini Gifts Distributors Ltd.' and 'Euro+ Shopping Channel'."